In [78]:
# Check CUDA availability (important for PyTorch & Transformers)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

# Load and shuffle your dataset
df = pd.read_csv("C:/Users/rugwe/OneDrive/Desktop/AI_LAB/Datasets/Final.csv")  # Replace with your path
df = shuffle(df, random_state=42)

X = df['text']  # Replace with actual text column name
y = df['labels']

# Train-Test Split (80/20)
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [80]:
# TF-IDF Features
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(max_features=5000)
X_train_tfidf = tfidf.fit_transform(X_train_raw)
X_test_tfidf = tfidf.transform(X_test_raw)

In [81]:
# Extract RoBERTa Embeddings (CUDA, Batched)
from transformers import RobertaTokenizer, RobertaModel
import torch
from tqdm import tqdm

tokenizer = RobertaTokenizer.from_pretrained("roberta-base")
roberta_model = RobertaModel.from_pretrained("roberta-base").to("cuda")
roberta_model.eval()

def get_roberta_embeddings(texts, batch_size=32):
    embeddings = []
    for i in tqdm(range(0, len(texts), batch_size)):
        batch = texts[i:i+batch_size]
        inputs = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512).to("cuda")
        with torch.no_grad():
            outputs = roberta_model(**inputs)
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
        embeddings.append(cls_embeddings.cpu())
    return torch.cat(embeddings).numpy()

X_train_bert = get_roberta_embeddings(X_train_raw.tolist())
X_test_bert = get_roberta_embeddings(X_test_raw.tolist())

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
100%|██████████| 13/13 [00:07<00:00,  1.78it/s]


In [82]:
# Combine All Feature Sets (Final Hybrid)
from scipy.sparse import hstack

# You can choose which features to include
# Only sparse + dense (use hstack only for sparse)
X_train_combined = hstack([X_train_tfidf])  # Add others like X_train_w2v, X_train_bert
X_test_combined = hstack([X_test_tfidf])    # Match same here

In [83]:
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score

# Random Forest
rf = RandomForestClassifier(n_estimators=200, random_state=42)
rf.fit(X_train_combined, y_train)
rf_preds = rf.predict(X_test_combined)
print("🌲 Random Forest Accuracy:", accuracy_score(y_test, rf_preds))
print(classification_report(y_test, rf_preds))

# XGBoost
xgb = XGBClassifier(eval_metric='logloss')
xgb.fit(X_train_combined, y_train)
xgb_preds = xgb.predict(X_test_combined)
print("⚡ XGBoost Accuracy:", accuracy_score(y_test, xgb_preds))
print(classification_report(y_test, xgb_preds))

# SVM
svm = SVC(kernel='linear')
svm.fit(X_train_combined, y_train)
svm_preds = svm.predict(X_test_combined)
print("🧩 SVM Accuracy:", accuracy_score(y_test, svm_preds))
print(classification_report(y_test, svm_preds))

🌲 Random Forest Accuracy: 0.9975
              precision    recall  f1-score   support

           0       1.00      0.99      1.00       197
           1       1.00      1.00      1.00       203

    accuracy                           1.00       400
   macro avg       1.00      1.00      1.00       400
weighted avg       1.00      1.00      1.00       400

⚡ XGBoost Accuracy: 0.9825
              precision    recall  f1-score   support

           0       0.98      0.98      0.98       197
           1       0.98      0.99      0.98       203

    accuracy                           0.98       400
   macro avg       0.98      0.98      0.98       400
weighted avg       0.98      0.98      0.98       400

🧩 SVM Accuracy: 0.9925
              precision    recall  f1-score   support

           0       0.99      0.99      0.99       197
           1       0.99      1.00      0.99       203

    accuracy                           0.99       400
   macro avg       0.99      0.99      0.99  

In [84]:
# Save Everything
import joblib
joblib.dump(tfidf, "models/tfidf_vectorizer.pkl")
joblib.dump(w2v_model, "models/w2v_model.pkl")
joblib.dump(rf, "models/rf_model.pkl")
joblib.dump(xgb, "models/xgb_model.pkl")
joblib.dump(svm, "models/svm_model.pkl")

['models/svm_model.pkl']

In [85]:
# Cross-Validation
from sklearn.model_selection import cross_val_score
import numpy as np

# Cross-validation for Random Forest
rf_cv_scores = cross_val_score(rf, X_train_combined, y_train, cv=5, scoring='accuracy')
print(f"🌲 Random Forest Cross-Validation Accuracy: {np.mean(rf_cv_scores):.4f} ± {np.std(rf_cv_scores):.4f}")

# Cross-validation for XGBoost
xgb_cv_scores = cross_val_score(xgb, X_train_combined, y_train, cv=5, scoring='accuracy')
print(f"⚡ XGBoost Cross-Validation Accuracy: {np.mean(xgb_cv_scores):.4f} ± {np.std(xgb_cv_scores):.4f}")

# Cross-validation for SVM
svm_cv_scores = cross_val_score(svm, X_train_combined, y_train, cv=5, scoring='accuracy')
print(f"🧩 SVM Cross-Validation Accuracy: {np.mean(svm_cv_scores):.4f} ± {np.std(svm_cv_scores):.4f}")

🌲 Random Forest Cross-Validation Accuracy: 0.9881 ± 0.0050
⚡ XGBoost Cross-Validation Accuracy: 0.9838 ± 0.0085
🧩 SVM Cross-Validation Accuracy: 0.9875 ± 0.0052


In [86]:
# Checking for Overfitting
# Checking train vs test accuracy for Random Forest
rf_train_accuracy = rf.score(X_train_combined, y_train)
rf_test_accuracy = accuracy_score(y_test, rf_preds)

print(f"🌲 Random Forest - Train Accuracy: {rf_train_accuracy:.4f}")
print(f"🌲 Random Forest - Test Accuracy: {rf_test_accuracy:.4f}")

# Checking train vs test accuracy for XGBoost
xgb_train_accuracy = xgb.score(X_train_combined, y_train)
xgb_test_accuracy = accuracy_score(y_test, xgb_preds)

print(f"⚡ XGBoost - Train Accuracy: {xgb_train_accuracy:.4f}")
print(f"⚡ XGBoost - Test Accuracy: {xgb_test_accuracy:.4f}")

# Checking train vs test accuracy for SVM
svm_train_accuracy = svm.score(X_train_combined, y_train)
svm_test_accuracy = accuracy_score(y_test, svm_preds)

print(f"🧩 SVM - Train Accuracy: {svm_train_accuracy:.4f}")
print(f"🧩 SVM - Test Accuracy: {svm_test_accuracy:.4f}")

🌲 Random Forest - Train Accuracy: 1.0000
🌲 Random Forest - Test Accuracy: 0.9975
⚡ XGBoost - Train Accuracy: 1.0000
⚡ XGBoost - Test Accuracy: 0.9825
🧩 SVM - Train Accuracy: 0.9988
🧩 SVM - Test Accuracy: 0.9925


In [87]:
# Load Pretrained RoBERTa Model & Tokenizer
from transformers import RobertaTokenizer, RobertaForSequenceClassification
import torch

# Load tokenizer and model
tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
model = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2)  # binary classification
model.to('cuda')  # Ensure to use GPU if available

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [106]:
# Prepare the Dataset
from datasets import Dataset
import pandas as pd

# Load your cleaned CSV
df = pd.read_csv("C:/Users/rugwe/OneDrive/Desktop/AI_LAB/Datasets/Final.csv")  # Or provide full path if needed

# Convert to HuggingFace Dataset
dataset = Dataset.from_pandas(df)

# Optional: Show a few samples
print(dataset[0])

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Set format for PyTorch
tokenized_datasets.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])

tokenized_datasets.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels"]
)

{'text': 'Dear Michael, I hope this message finds you well. We are reaching out to you with an urgent request regarding your account verification. As part of our commitment to ensuring the security of our users, we kindly ask you to verify your account at your earliest convenience. By completing this simple verification process, you not only enhance the security of your account but also contribute to making our community safer for everyone. Your cooperation in this matter is greatly appreciated. Please click [here](https://account-verification-link1.com) to proceed with the verification process. Thank you for your prompt attention to this matter. Best regards, Sarah Johnson Customer Support Team', 'labels': 0}


Map: 100%|██████████| 2000/2000 [00:01<00:00, 1368.35 examples/s]


In [107]:
# Train-test split (80-20)
train_dataset = tokenized_datasets.shuffle(seed=42).select([i for i in list(range(int(0.8 * len(tokenized_datasets))))])
test_dataset = tokenized_datasets.select([i for i in list(range(int(0.8 * len(tokenized_datasets)), len(tokenized_datasets)))])

In [108]:
# Define DataLoader
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=8)

In [109]:
# Define Optimizer & Scheduler
from transformers import AdamW, get_linear_schedule_with_warmup

# Optimizer and Scheduler
optimizer = AdamW(model.parameters(), lr=5e-5, eps=1e-8)
total_steps = len(train_dataloader) * 3  # Train for 3 epochs
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0, 
                                            num_training_steps=total_steps)

c:\Users\rugwe\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:640: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [110]:
# Fine-tuning Loop
from sklearn.metrics import accuracy_score

# Training loop
epochs = 3  # number of epochs
for epoch in range(epochs):
    model.train()  # Set model to train mode
    total_loss = 0
    correct_preds = 0
    total_preds = 0
    for batch in train_dataloader:
        # Move data to GPU if available
        batch = {k: v.to(device) for k, v in batch.items() if isinstance(v, torch.Tensor)}
        
        # Forward pass
        print(batch.keys())  # Debugging: Check if 'input_ids' is in batch
        outputs = model(**batch)
        loss = outputs.loss
        logits = outputs.logits
        
        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        scheduler.step()
        
        total_loss += loss.item()
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        correct_preds += (preds == batch['labels']).sum().item()
        total_preds += batch['labels'].size(0)

    # Calculate training loss and accuracy
    avg_train_loss = total_loss / len(train_dataloader)
    train_accuracy = correct_preds / total_preds

    print(f"Epoch {epoch + 1}/{epochs} - Loss: {avg_train_loss:.4f} - Accuracy: {train_accuracy:.4f}")

dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_mask'])
dict_keys(['labels', 'input_ids', 'attention_m

In [112]:
# Evaluation on the Test Set
model.eval()  # Set model to evaluation mode
correct_preds = 0
total_preds = 0

with torch.no_grad():
    for batch in test_dataloader:
        batch = {k: v.to('cuda') for k, v in batch.items()}
        
        outputs = model(**batch)
        logits = outputs.logits
        
        # Calculate accuracy
        preds = torch.argmax(logits, dim=-1)
        correct_preds += (preds == batch['labels']).sum().item()
        total_preds += batch['labels'].size(0)

test_accuracy = correct_preds / total_preds
print(f"Test Accuracy: {test_accuracy:.4f}")

Test Accuracy: 1.0000


In [113]:
# Save the Fine-Tuned Model
model.save_pretrained("models/roberta_finetuned")
tokenizer.save_pretrained("models/roberta_finetuned_tokenizer")

('models/roberta_finetuned_tokenizer\\tokenizer_config.json',
 'models/roberta_finetuned_tokenizer\\special_tokens_map.json',
 'models/roberta_finetuned_tokenizer\\vocab.json',
 'models/roberta_finetuned_tokenizer\\merges.txt',
 'models/roberta_finetuned_tokenizer\\added_tokens.json')

In [ ]:
# LIME Setup
from lime.lime_text import LimeTextExplainer
import numpy as np

# Class names
class_names = ["Not Phishing", "Phishing"]

# LIME Explainer
lime_explainer = LimeTextExplainer(class_names=class_names)

# Prediction wrapper for LIME
def lime_predict(texts):
    encodings = tokenizer(texts, return_tensors='pt', padding=True, truncation=True, max_length=512)
    encodings = {k: v.to(device) for k, v in encodings.items()}

    with torch.no_grad():
        outputs = model(**encodings)
        probs = torch.nn.functional.softmax(outputs.logits, dim=1)

    return probs.cpu().numpy()

In [123]:
# Explain and Visualize with LIME
def explain_with_lime(text):
    exp = lime_explainer.explain_instance(text, lime_predict, num_features=10)

    prediction = lime_predict([text])[0]
    predicted_class = int(np.argmax(prediction))
    confidence = float(prediction[predicted_class])

    # Simple threshold for manual review
    needs_manual_review = confidence < 0.7

    return {
        "predicted_label": class_names[predicted_class],
        "confidence": round(confidence, 2),
        "needs_manual_review": needs_manual_review,
        "lime_explanation": exp
    }

In [126]:
# Token-Level Highlighting in Streamlit (Optional)
import streamlit as st

def display_lime_explanation(exp):
    highlighted_text = exp.as_html()
    st.markdown("#### 🔎 Token-level Explanation (LIME)")
    # st.components.v1.html(highlighted_text, height=400, scrolling=True)
    print(exp.as_list())  # View raw token importances